# LiricAI

Aqui é mostrado um pequeno projecto em que se usa uma Recurrent Neural Network (segundo https://gilberttanner.com/blog/generating-text-using-a-recurrent-neuralnetwork) para criar a letra de uma música a partir de um determinado conjunto de letras já existente. 

## Os dados

Os ficheiros utilizados encontram-se disponíveis no repositório de Github como ficheiros .txt

## Bibliotecas a usar

Como estou a seguir a ligação acima referida, Keras e Tensorflow.

## Tratamento de dados

In [7]:
from os import listdir
from os.path import isfile, join
import sys
import collections, functools, operator
import numpy as np
import random
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.callbacks import LambdaCallback, ModelCheckpoint, ReduceLROnPlateau

Contar os caracteres presentes nas letras:

In [8]:
dataset_path = "./dataset"
files_in_dataset = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]
char_indices_list = []
indices_char_list = []

total_lyrics = ""
for filename in files_in_dataset:
    filepath = dataset_path +"/"+filename
    with open(filepath, 'r',encoding='utf8') as file:

        text = file.read().lower()
      
        total_lyrics += "\n\n"+text 

chars = sorted(list(set(total_lyrics))) # getting all unique chars

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)

{'\t': 0, '\n': 1, ' ': 2, '!': 3, '"': 4, '%': 5, "'": 6, '(': 7, ')': 8, '+': 9, ',': 10, '-': 11, '.': 12, '/': 13, '0': 14, '1': 15, '2': 16, '3': 17, '4': 18, '5': 19, '6': 20, '7': 21, '8': 22, '9': 23, ':': 24, ';': 25, '?': 26, '[': 27, ']': 28, 'a': 29, 'b': 30, 'c': 31, 'd': 32, 'e': 33, 'f': 34, 'g': 35, 'h': 36, 'i': 37, 'j': 38, 'k': 39, 'l': 40, 'm': 41, 'n': 42, 'o': 43, 'p': 44, 'q': 45, 'r': 46, 's': 47, 't': 48, 'u': 49, 'v': 50, 'w': 51, 'x': 52, 'y': 53, 'z': 54, '{': 55, '}': 56, 'ª': 57, '«': 58, '´': 59, '»': 60, 'à': 61, 'á': 62, 'â': 63, 'ã': 64, 'ç': 65, 'è': 66, 'é': 67, 'ê': 68, 'í': 69, 'ñ': 70, 'ó': 71, 'ô': 72, 'õ': 73, 'ú': 74, 'ü': 75, '–': 76, '‘': 77, '’': 78, '“': 79, '”': 80, '…': 81}


In [9]:
max_len = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(total_lyrics) - max_len, step):
    sentences.append(total_lyrics[i: i + max_len])
    next_chars.append(total_lyrics[i + max_len])

x = np.zeros((len(sentences), max_len, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

## Recurrent Neural Network

In [10]:
model = Sequential()
model.add(LSTM(256, input_shape=(max_len, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

C:\Users\danie\anaconda3\lib\site-packages\keras\optimizer_v2\rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [11]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - max_len - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + max_len]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [12]:
filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [13]:
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)

callbacks = [print_callback, checkpoint, reduce_lr]

## Treinar a rede

In [14]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks, verbose=0)


----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ico não peco
lá em baixo ficava marreco
"
ico não peco
lá em baixo ficava marreco

não te de como a cora de meu dia a tua a tua camo a sorra a falar
na airana

e eu no conse está não se a cora não sem como a tua meu não sei a cora de meu nas a altar
e a minha mande a sorra a saber
se não tente a cora de mara a sorrar
e a tua de meu dia de meu não ser a sorra a altar
e a altar
e a minha mara a altar
e a tua meu não ser a sorra a mara está no cante a altar
a vida a tua a terra a 
----- diversity: 0.5
----- Generating with seed: "ico não peco
lá em baixo ficava marreco
"
ico não peco
lá em baixo ficava marreco
de minada e vou pera a rua de mara de cilado de sonte amora sinte re para a altar
com uma mas a sorra com eu sei a tuande alguém a nunca na altar
tudo a trada a mas de como mas a dizer

no andar a destiro o solte fantar

de antero de tempo o cantio de sol e estante está como a luar
de a minha ter

o que é tão compo -lhe um barga lenção ão nem continum samba nem bem

estou sorrenti
do de as
pareco como é bem
atrasta vardo à solusiro

a tanca bilu a manhar, amor sem amor
adega trá landa na tinar certo l
----- diversity: 1.2
----- Generating with seed: "vos
mas só sei crescer

leio o topo da e"
vos
mas só sei crescer

leio o topo da embatal
no volto solosa é paianuma para pregresete
nar(ídosaruml
o, surás salgar
tu quemo bem
má de ti
segue o que te é no restoupre, outraconderbeços
é leu, tudo pago apregada

canhou salgas pa ória tão dor.ptaco-me acontirah,
tãohmem era uma ãor
óireteha coma olha alta esnado
raja lenhuemada. vêtsa atradadrapas
sãoiro, pegcuiro, pencolh, tama banta, lha corpo um
soar

como s quem amor fom ensar, 

Epoch 00004: loss improved from 1.62571 to 1.50110, saving model to weights.hdf5

----- Generating text after Epoch: 4
----- diversity: 0.2
----- Generating with seed: "i crescer

eu só me caibo cá dentro
mas "
i crescer

eu só me caibo cá dentro
mas não pon

In [16]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - max_len - 1)
    generated = ''
    sentence = text[start_index: start_index + max_len]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, max_len, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [17]:
import datetime
number_of_files = 5
numberchar = 250
date = datetime.datetime.now().date()

for counter in range(0, number_of_files):
    songFileName = "results/output_"+str(date)+"_"+str(numberchar)+"_"+str(counter)+".txt"
    with open(songFileName, 'w') as f:
        f.write(generate_text(numberchar, 0.2))

#print(generate_text(500, 0.2))